In [1]:
# Import libraries
from PIL import Image
import numpy as np

# Function to load the image and convert to NumPy array
def load_image(image_path):
    img = Image.open(image_path).convert('RGB')
    return np.array(img)


In [37]:
# Function to detect black pixels, count neighbors, modify ex-black pixels, and count changes
def detect_and_modify_black_pixels(image_array, rarity_group1=0.005, rarity_group2=0.002, rarity_group3=0.001):
    black_pixel_count = 0
    black_with_neighbors_count = 0
    changed_pixel_count_group1 = 0
    changed_pixel_count_group2 = 0
    changed_pixel_count_group3 = 0

    # Direction-specific counters
    direction_count_group1 = {"top": 0, "bottom": 0, "left": 0, "right": 0}
    direction_count_group2 = {"top": 0, "bottom": 0, "left": 0, "right": 0}
    direction_count_group3 = {"top": 0, "bottom": 0, "left": 0, "right": 0}
    
    height, width, _ = image_array.shape
    
    # Clone the array for modifications
    modified_array = image_array.copy()
    
    # Track black pixels that could be modified later
    black_pixels_to_modify = []

    for y in range(height):
        for x in range(width):
            # Check if the pixel is black
            if all(image_array[y, x] == [0, 0, 0]):
                black_pixel_count += 1
                # Check neighbors (4-connectivity)
                neighbors = [
                    (y-1, x, "top"), 
                    (y+1, x, "bottom"), 
                    (y, x-1, "left"), 
                    (y, x+1, "right")  # top, bottom, left, right
                ]
                has_non_black_neighbor = False
                neighbor_direction = None
                for ny, nx, direction in neighbors:
                    if 0 <= ny < height and 0 <= nx < width:
                        if not all(image_array[ny, nx] == [0, 0, 0]):  # Non-black neighbor
                            black_with_neighbors_count += 1
                            has_non_black_neighbor = True
                            neighbor_direction = direction
                            break
                # If it has a non-black neighbor, consider it for modification
                if has_non_black_neighbor:
                    black_pixels_to_modify.append((y, x, neighbor_direction))
    
    # Split eligible pixels into groups for modifications
    num_to_modify_group1 = int(len(black_pixels_to_modify) * rarity_group1)
    num_to_modify_group2 = int(len(black_pixels_to_modify) * rarity_group2)
    num_to_modify_group3 = int(len(black_pixels_to_modify) * rarity_group3)
    
    # Randomly select pixels for each group, ensuring no overlap
    modified_pixels_group1 = np.random.choice(len(black_pixels_to_modify), num_to_modify_group1, replace=False)
    remaining_pixels_after_group1 = set(range(len(black_pixels_to_modify))) - set(modified_pixels_group1)
    modified_pixels_group2 = np.random.choice(list(remaining_pixels_after_group1), num_to_modify_group2, replace=False)
    remaining_pixels_after_group2 = remaining_pixels_after_group1 - set(modified_pixels_group2)
    modified_pixels_group3 = np.random.choice(list(remaining_pixels_after_group2), num_to_modify_group3, replace=False)
    
    # Apply modifications for group 1
    for idx in modified_pixels_group1:
        y, x, direction = black_pixels_to_modify[idx]
        if direction == "top":
            modified_array[y, x] = [36, 229, 191]  # Color for top-connected
            direction_count_group1["top"] += 1
        elif direction == "bottom":
            modified_array[y, x] = [36, 229, 181]  # Color for bottom-connected
            direction_count_group1["bottom"] += 1
        elif direction == "left":
            modified_array[y, x] = [36, 229, 201]  # Color for left-connected
            direction_count_group1["left"] += 1
        elif direction == "right":
            modified_array[y, x] = [36, 229, 211]  # Color for right-connected
            direction_count_group1["right"] += 1
        changed_pixel_count_group1 += 1

    # Apply modifications for group 2
    for idx in modified_pixels_group2:
        y, x, direction = black_pixels_to_modify[idx]
        if direction == "top":
            modified_array[y, x] = [36, 210, 170]  # Color for top-connected
            direction_count_group2["top"] += 1
        elif direction == "bottom":
            modified_array[y, x] = [36, 200, 170]  # Color for bottom-connected
            direction_count_group2["bottom"] += 1
        elif direction == "left":
            modified_array[y, x] = [36, 220, 170]  # Color for left-connected
            direction_count_group2["left"] += 1
        elif direction == "right":
            modified_array[y, x] = [36, 230, 170]  # Color for right-connected
            direction_count_group2["right"] += 1
        changed_pixel_count_group2 += 1

    # Apply modifications for group 3
    for idx in modified_pixels_group3:
        y, x, direction = black_pixels_to_modify[idx]
        if direction == "top":
            modified_array[y, x] = [50, 200, 180]  # Color for top-connected
            direction_count_group3["top"] += 1
        elif direction == "bottom":
            modified_array[y, x] = [40, 200, 180]  # Color for bottom-connected
            direction_count_group3["bottom"] += 1
        elif direction == "left":
            modified_array[y, x] = [60, 200, 180]  # Color for left-connected
            direction_count_group3["left"] += 1
        elif direction == "right":
            modified_array[y, x] = [70, 200, 180]  # Color for right-connected
            direction_count_group3["right"] += 1
        changed_pixel_count_group3 += 1

    return (
        black_pixel_count,
        black_with_neighbors_count,
        changed_pixel_count_group1,
        direction_count_group1,
        changed_pixel_count_group2,
        direction_count_group2,
        changed_pixel_count_group3,
        direction_count_group3,
        modified_array,
    )


In [38]:
# Load the image
image_path = "Level1/10.png"  # Replace with your image file
image_array = load_image(image_path)

# Detect and modify black pixels with three rarity levels
rarity_group1 = 0.03  # Adjust rarity for group 1
rarity_group2 = 0.04  # Adjust rarity for group 2
rarity_group3 = 0.005  # Adjust rarity for group 3

(
    total_black,
    black_with_neighbors,
    changed_pixel_count_group1,
    direction_count_group1,
    changed_pixel_count_group2,
    direction_count_group2,
    changed_pixel_count_group3,
    direction_count_group3,
    modified_array,
) = detect_and_modify_black_pixels(image_array, rarity_group1, rarity_group2, rarity_group3)

# Save the modified image
modified_image = Image.fromarray(modified_array)
modified_image.save("Level1/1.png")

# Print results
print(f"Total black pixels: {total_black}")
print(f"Black pixels with non-black neighbors: {black_with_neighbors}")
print(f"Modified pixels for Group 1: {changed_pixel_count_group1}")
print(f"Group 1 modified pixels by direction: {direction_count_group1}")
print(f"Modified pixels for Group 2: {changed_pixel_count_group2}")
print(f"Group 2 modified pixels by direction: {direction_count_group2}")
print(f"Modified pixels for Group 3: {changed_pixel_count_group3}")
print(f"Group 3 modified pixels by direction: {direction_count_group3}")


Total black pixels: 46004
Black pixels with non-black neighbors: 4441
Modified pixels for Group 1: 133
Group 1 modified pixels by direction: {'top': 39, 'bottom': 37, 'left': 35, 'right': 22}
Modified pixels for Group 2: 177
Group 2 modified pixels by direction: {'top': 50, 'bottom': 47, 'left': 40, 'right': 40}
Modified pixels for Group 3: 22
Group 3 modified pixels by direction: {'top': 10, 'bottom': 6, 'left': 2, 'right': 4}
